In [186]:
import pandas as pd
import numpy as np

In [181]:
def new_simlib(template_simlib_fname, output_fname, days_span, sample_rate_days, whitespace=True):
    i=0
    docrows = []
    with open(template_simlib_fname, 'r') as template:
        for line in template:
            docrows.append(line)
            if 'MJD' in line:
                skiprows= i+1
                break
            i+=1

    hdr        = ["MJD", "IDEXPT", "FLT", "CCD GAIN", "CCD NOISE", "SKYSIG", "PSF1", "PSF2", "PSF 2/1 RATIO", "ZPTAVG", "ZPTERR", "MAG"]
    simlib_old = pd.read_csv(template_simlib_fname, names=hdr, skiprows=skiprows, delim_whitespace=whitespace, skipfooter=2)

    numsteps   = int(days_span/sample_rate_days)
    mjdsteps   = min(simlib_old['MJD'])+np.linspace(0,days_span,numsteps)
    mjd        = []
    for m in range(len(mjdsteps)):
        for n in range(6):
            mjd.append(mjdsteps[m])
    idexpt     = range(min(simlib_old['IDEXPT']),min(simlib_old['IDEXPT'])+len(mjd))
    
    cvals = []
    for h in hdr[3:]:
        val = simlib_old[h][0]
        cvals.append(val)
    with open(output_fname, 'w') as newfile:
        for row in docrows:
            newfile.write(row)
        for c in range(len(mjd)):
            m = c%6
            bands = ['u','g','r','i','z','Y']
            newfile.write("S: {:0.3f}    {:d}   {:s}  {:.2f}  {:.2f}  {:.1f}  {:.2f} {:.2f} {:.3f}  {:.2f}  {:.2f}  {:d}\n".format(mjd[c],idexpt[c],
                            bands[m], cvals[0],cvals[1],cvals[2],cvals[3],cvals[4],cvals[5],cvals[6],cvals[7],cvals[8]))
            if m==5:
                newfile.write("\n")

In [187]:
new_simlib("/global/cfs/cdirs/lsst/groups/SN/snana/SURVEYS/LSST/ROOT/PLASTICC_DEV/survey_config/LSST_IDEAL_0.1DAY.SIMLIB", "/global/cfs/cdirs/lsst/groups/SN/snana/SURVEYS/LSST/ROOT/PLASTICC_DEV/survey_config/KN_0.1DAY.SIMLIB", 10, 0.1)

/global/common/software/lsst/common/miniconda/prod/envs/desc/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [183]:
hdr = ["MJD", "IDEXPT", "FLT", "CCD GAIN", "CCD NOISE", "SKYSIG", "PSF1", "PSF2", "PSF 2/1 RATIO", "ZPTAVG", "ZPTERR", "MAG"]
simlib = pd.read_csv("/global/cfs/cdirs/lsst/groups/SN/snana/SURVEYS/LSST/ROOT/simlibs/LSST_IDEAL_0.1DAY.SIMLIB", skiprows=8, names=hdr, delim_whitespace=True, skipfooter=2)
simlib.iloc[:10]

,MJD,IDEXPT,FLT,CCD GAIN,CCD NOISE,SKYSIG,PSF1,PSF2,PSF 2/1 RATIO,ZPTAVG,ZPTERR,MAG
S:,53100.0,10001,u,1.0,1.0,40.0,1.0,0.0,0.0,35.0,0.0,99
S:,53100.0,10002,g,1.0,1.0,40.0,1.0,0.0,0.0,35.0,0.0,99
S:,53100.0,10003,r,1.0,1.0,40.0,1.0,0.0,0.0,35.0,0.0,99
S:,53100.0,10004,i,1.0,1.0,40.0,1.0,0.0,0.0,35.0,0.0,99
S:,53100.0,10005,z,1.0,1.0,40.0,1.0,0.0,0.0,35.0,0.0,99
S:,53100.0,10006,Y,1.0,1.0,40.0,1.0,0.0,0.0,35.0,0.0,99
S:,53100.1,10007,u,1.0,1.0,40.0,1.0,0.0,0.0,35.0,0.0,99
S:,53100.1,10008,g,1.0,1.0,40.0,1.0,0.0,0.0,35.0,0.0,99
S:,53100.1,10009,r,1.0,1.0,40.0,1.0,0.0,0.0,35.0,0.0,99
S:,53100.1,10010,i,1.0,1.0,40.0,1.0,0.0,0.0,35.0,0.0,99


In [184]:
simlib['MJD']

S:    53100.0
S:    53100.0
S:    53100.0
S:    53100.0
S:    53100.0
       ...   
S:    53120.0
S:    53120.0
S:    53120.0
S:    53120.0
S:    53120.0
Name: MJD, Length: 1206, dtype: float64

In [185]:
max(simlib['MJD'])-min(simlib['MJD'])

20.0